# For this lesson, we will use the libraries listed below.

In [1]:
import math
import collections

import numpy as np
import pandas as pd

# Reading a URL with Pandas

## We will be reading in the table "Player Standard Batting" From the URL: https://www.baseball-reference.com/leagues/MLB/1987-standard-batting.shtml.

In [2]:
#because of the site changes, you must modify then save the table as a new link
url = "http://bbref.com/pi/shareit/w9s78"
Players = pd.read_html(url)[0]

#Keeping it simple, we'll work with a few columns (up to SO, strikeouts)
Players = Players.iloc[:, 1:19]
Players.head()

,Player,Age,Team,Lg,WAR,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO
0,Andre Dawson,32,CHC,NL,4.0,153,662,621,90,178,24,2,49,137,11,3,32,103
1,Mark McGwire,23,OAK,AL,5.1,151,641,557,97,161,28,4,49,118,1,1,71,131
2,George Bell,27,TOR,AL,5.0,156,667,610,111,188,32,4,47,134,5,1,39,75
3,Dale Murphy,31,ATL,NL,7.7,159,693,566,115,167,27,1,44,105,16,6,115,136
4,Darryl Strawberry*,25,NYM,NL,6.4,154,640,532,108,151,32,5,39,104,36,12,97,122


In [70]:
Players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  912 non-null    object
 1   Age     911 non-null    object
 2   Team    911 non-null    object
 3   Lg      911 non-null    object
 4   WAR     903 non-null    object
 5   G       911 non-null    object
 6   PA      911 non-null    object
 7   AB      911 non-null    object
 8   R       911 non-null    object
 9   H       911 non-null    object
 10  2B      911 non-null    object
 11  3B      911 non-null    object
 12  HR      911 non-null    object
 13  RBI     911 non-null    object
 14  SB      911 non-null    object
 15  CS      911 non-null    object
 16  BB      911 non-null    object
 17  SO      911 non-null    object
dtypes: object(18)
memory usage: 128.4+ KB


## Cleaning erronious rows and characters.

In [71]:
#run this first time through
Players = Players.replace({'AB':np.nan}).dropna() #Removes extra column title lines

special_characters = ['*', '#', '@', '&']

for i in special_characters:
    Players.Player = Players.Player.str.replace(i, '') #Removes extraneous characters.

#not sure why this stopped working
# Players.Player = Players.Player.str.replace('[*,#,@,&]', '')

Players

,Player,Age,Team,Lg,WAR,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO
0,Andre Dawson,32,CHC,NL,4.0,153,662,621,90,178,24,2,49,137,11,3,32,103
1,Mark McGwire,23,OAK,AL,5.1,151,641,557,97,161,28,4,49,118,1,1,71,131
2,George Bell,27,TOR,AL,5.0,156,667,610,111,188,32,4,47,134,5,1,39,75
3,Dale Murphy,31,ATL,NL,7.7,159,693,566,115,167,27,1,44,105,16,6,115,136
4,Darryl Strawberry,25,NYM,NL,6.4,154,640,532,108,151,32,5,39,104,36,12,97,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,Butch Wynegar,31,CAL,AL,0.0,31,102,92,4,19,2,0,0,5,0,0,9,13
907,Curt Young,27,OAK,AL,0.0,1,1,1,0,0,0,0,0,0,0,0,0,0
908,Matt Young,28,LAD,NL,0.0,47,3,3,0,0,0,0,0,0,0,0,0,2
909,Steve Ziem,25,ATL,NL,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0


In [72]:
#Wait to run this part until second time through

#prompted ChatGPT for this using hte prompt "how to remove special characters from the names column in Pandas"

Players.Player = Players.Player.str.replace(r'[^a-zA-Z\s]', '', regex=True)

# r'[^a-zA-Z\s]' is a regular expression that matches any character that is not a letter (both uppercase and lowercase) or a whitespace.

Players.head()

,Player,Age,Team,Lg,WAR,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO
0,Andre Dawson,32,CHC,NL,4.0,153,662,621,90,178,24,2,49,137,11,3,32,103
1,Mark McGwire,23,OAK,AL,5.1,151,641,557,97,161,28,4,49,118,1,1,71,131
2,George Bell,27,TOR,AL,5.0,156,667,610,111,188,32,4,47,134,5,1,39,75
3,Dale Murphy,31,ATL,NL,7.7,159,693,566,115,167,27,1,44,105,16,6,115,136
4,Darryl Strawberry,25,NYM,NL,6.4,154,640,532,108,151,32,5,39,104,36,12,97,122


## Cleaning duplicates

In [73]:
#Note that this keeps the first entry and leaves the
# (ex. Delete the the 2nd and 3rd entry for Bill Almon)
Players[Players['Player'] == 'Bill Almon']

,Player,Age,Team,Lg,WAR,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO
511,Bill Almon,34,2TM,NL,-0.2,68,83,74,13,17,4,0,0,5,1,0,9,21
512,Bill Almon,34,PIT,NL,-0.3,19,21,20,5,4,1,0,0,1,0,0,1,5
513,Bill Almon,34,NYM,NL,0.1,49,62,54,8,13,3,0,0,4,1,0,8,16


In [74]:
Players = Players.drop_duplicates(subset=['Player'])
Players

,Player,Age,Team,Lg,WAR,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO
0,Andre Dawson,32,CHC,NL,4.0,153,662,621,90,178,24,2,49,137,11,3,32,103
1,Mark McGwire,23,OAK,AL,5.1,151,641,557,97,161,28,4,49,118,1,1,71,131
2,George Bell,27,TOR,AL,5.0,156,667,610,111,188,32,4,47,134,5,1,39,75
3,Dale Murphy,31,ATL,NL,7.7,159,693,566,115,167,27,1,44,105,16,6,115,136
4,Darryl Strawberry,25,NYM,NL,6.4,154,640,532,108,151,32,5,39,104,36,12,97,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
906,Butch Wynegar,31,CAL,AL,0.0,31,102,92,4,19,2,0,0,5,0,0,9,13
907,Curt Young,27,OAK,AL,0.0,1,1,1,0,0,0,0,0,0,0,0,0,0
908,Matt Young,28,LAD,NL,0.0,47,3,3,0,0,0,0,0,0,0,0,0,2
909,Steve Ziem,25,ATL,NL,0.0,2,0,0,0,0,0,0,0,0,0,0,0,0


In [75]:
Players[Players['Player'] == 'Bill Almon']

,Player,Age,Team,Lg,WAR,G,PA,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO
511,Bill Almon,34,2TM,NL,-0.2,68,83,74,13,17,4,0,0,5,1,0,9,21


## Data Type Conversions

In [76]:
Players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 789 entries, 0 to 910
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Player  789 non-null    object
 1   Age     789 non-null    object
 2   Team    789 non-null    object
 3   Lg      789 non-null    object
 4   WAR     789 non-null    object
 5   G       789 non-null    object
 6   PA      789 non-null    object
 7   AB      789 non-null    object
 8   R       789 non-null    object
 9   H       789 non-null    object
 10  2B      789 non-null    object
 11  3B      789 non-null    object
 12  HR      789 non-null    object
 13  RBI     789 non-null    object
 14  SB      789 non-null    object
 15  CS      789 non-null    object
 16  BB      789 non-null    object
 17  SO      789 non-null    object
dtypes: object(18)
memory usage: 117.1+ KB


In [79]:
#do the easy one first, one column (WAR) needs to be converted to a float
Players['WAR'] = pd.to_numeric(Players['WAR'])
type(Players['WAR'][0])

C:\Users\1187483510E\AppData\Local\Temp\ipykernel_21172\974953656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Players['WAR'] = pd.to_numeric(Players['WAR'])


numpy.float64

In [80]:
#we know that everything from Games over should be converted to an integer
#locate what col index Games (G) starts
Players.columns.get_loc('G')

#We can get the set of column names using our slicing
Players.columns[5:]

#index has a function called "insert" that we can make use of to grab our last column
Players[Players.columns[5:].insert(0, 'Age')].astype(int)

#gotta save it
Players[Players.columns[5:].insert(0, 'Age')] = Players[Players.columns[5:].insert(0, 'Age')].astype(int)

C:\Users\1187483510E\AppData\Local\Temp\ipykernel_21172\2534088749.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Players[Players.columns[5:].insert(0, 'Age')] = Players[Players.columns[5:].insert(0, 'Age')].astype(int)


In [81]:
#Players[Players.columns[4:].insert(0,'Age')]= Players[Players.columns[4:].insert(0,'Age')].astype(int)
Players.info()

<class 'pandas.core.frame.DataFrame'>
Index: 789 entries, 0 to 910
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  789 non-null    object 
 1   Age     789 non-null    int32  
 2   Team    789 non-null    object 
 3   Lg      789 non-null    object 
 4   WAR     789 non-null    float64
 5   G       789 non-null    int32  
 6   PA      789 non-null    int32  
 7   AB      789 non-null    int32  
 8   R       789 non-null    int32  
 9   H       789 non-null    int32  
 10  2B      789 non-null    int32  
 11  3B      789 non-null    int32  
 12  HR      789 non-null    int32  
 13  RBI     789 non-null    int32  
 14  SB      789 non-null    int32  
 15  CS      789 non-null    int32  
 16  BB      789 non-null    int32  
 17  SO      789 non-null    int32  
dtypes: float64(1), int32(14), object(3)
memory usage: 106.3+ KB


## 5. Save edited data frame to a `.csv` file. Check file to ensure data is appropriately cleaned and ready for analysis. Note: The `.csv` file will need to be submitted with your `.ipynb` file.

In [82]:
Players.to_csv('Players.csv',index=False)